# Advent of Code Day 01
Coded in Db2 SQL. See [readme](https://github.com/ecrooks/aoc2021_SQL) for more details on how to use. 

Env setup - only needs to run once on each machine

In [ ]:
!pip install ipython-sql
!pip install ibm_db 
!pip install ibm_db_sa

Restart the Kernel if this is your first time installing the above. The next steps will fail unless you do this.

Import modules and load the SQL magic

In [3]:
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
import getpass

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Connect to the database. Change the values of user, host, and password to match your environment. For connection to a local host, use 'localhost' for the host name. Also change the port number and database name in the connection string.

In [4]:
user='db2inst1'
host='localhost'
dbname='aocdb'


password = getpass.getpass('Enter password for '+user+' on '+dbname)

%sql db2+ibm_db://$user:$password@$host:50000/$dbname

Enter password for db2inst1 on aocdb········


## Day 1, Part 1 Solution

### Problem statement: 

>As the submarine drops below the surface of the ocean, it automatically performs a sonar sweep of the nearby sea floor. On a small screen, the sonar sweep report (your puzzle input) appears: each line is a measurement of the sea floor depth as the sweep looks further and further away from the submarine.  
>
>For example, suppose you had the following report:  
>
>199  
>200  
>208  
>210  
>200  
>207  
>240  
>269  
>260  
>263  
>This report indicates that, scanning outward from the submarine, the sonar sweep found depths of 199, 200, 208, 210, >and so on.  
>
>The first order of business is to figure out how quickly the depth increases, just so you know what you're dealing with - you never know if the keys will get carried into deeper water by an ocean current or a fish or something.  
>
>To do this, count the number of times a depth measurement increases from the previous measurement. (There is no measurement before the first measurement.) In the example above, the changes are as follows:  
>
>199 (N/A - no previous measurement)  
>200 (increased)  
>208 (increased)  
>210 (increased)  
>200 (decreased)  
>207 (increased)  
>240 (increased)  
>269 (increased)  
>260 (decreased)  
>263 (increased)  
>In this example, there are 7 measurements that are larger than the previous measurement.  
>
>How many measurements are larger than the previous measurement?  
>

In [5]:
# quick view of the data
%sql select * from aoc_day01 fetch first 10 rows only

 * db2+ibm_db://db2inst1:***@localhost:50000/aocdb
Done.


seq,depth
1,199
2,200
3,208
4,210
5,200
6,207
7,240
8,269
9,260
10,263


### Final Solution to Part 1

In [17]:
%%sql with depth_diff as (select depth - lag(depth,1) over(order by seq asc) as diff
    from aoc_day01)
select count(*) as count 
    from depth_diff where diff > 0

 * db2+ibm_db://db2inst1:***@localhost:50000/aocdb
Done.


COUNT
1655


## Day 1, Part 2 Solution

### Problem statement: 
>Considering every single measurement isn't as useful as you expected: there's just too much noise in the data.  
>  
>Instead, consider sums of a three-measurement sliding window. Again considering the above example:  
>
```199  A      
200  A B    
208  A B C  
210    B C D
200  E   C D
207  E F   D
240  E F G  
269    F G H
260      G H
263        H```  


>Start by comparing the first and second three-measurement windows. The measurements in the first window are marked A (199, 200, 208); their sum is 199 + 200 + 208 = 607. The second window is marked B (200, 208, 210); its sum is 618. The sum of measurements in the second window is larger than the sum of the first, so this first comparison increased.  
>  
>Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum. So, compare A with B, then compare B with C, then C with D, and so on. Stop when there aren't enough measurements left to create a new three-measurement sum.  
>  
>In the above example, the sum of each three-measurement window is as follows:  

>A: 607 (N/A - no previous sum)  
>B: 618 (increased)  
>C: 618 (no change)  
>D: 617 (decreased)  
>E: 647 (increased)  
>F: 716 (increased)  
>G: 769 (increased)  
>H: 792 (increased)  
>In this example, there are 5 sums that are larger than the previous sum.  
>  
>Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?  



In [25]:
%%sql select depth
    ,lag(depth,1) over(order by seq asc)
    , lag(depth,2) over(order by seq asc)
    , depth + lag(depth,1) over(order by seq asc) + lag(depth,2) over(order by seq asc)
    from aoc_day01

 * db2+ibm_db://db2inst1:***@localhost:50000/aocdb
Done.


depth,2,3,4
199,None,None,None
200,199,None,None
208,200,199,607
210,208,200,618
200,210,208,618
207,200,210,617
240,207,200,647
269,240,207,716
260,269,240,769
263,260,269,792


Fairly manual approach, would not scale to larger windows, but should get the right answer

In [33]:
%%sql with depth3 as (select seq, depth + lag(depth,1) over(order by seq asc) + lag(depth,2) over(order by seq asc) as depth3
        from aoc_day01)
    ,depth3_diff as (select depth3 - (lag(depth3,1) over(order by seq asc)) as diff
        from depth3)
    select count(*) 
        from depth3_diff
        where diff > 0

 * db2+ibm_db://db2inst1:***@localhost:50000/aocdb
Done.


1
5


In [46]:
%%sql select 
    case when count(depth) over(order by seq asc ROWS 2 PRECEDING) = 3
        then sum(depth) over(order by seq asc ROWS 2 PRECEDING) 
        else null
        end as diff
    from aoc_day01

 * db2+ibm_db://db2inst1:***@localhost:50000/aocdb
Done.


diff
None
None
607
618
618
617
647
716
769
792


In [47]:
%%sql with depth3 as (select 
    case when count(depth) over(order by seq asc ROWS 2 PRECEDING) = 3
        then sum(depth) over(order by seq asc ROWS 2 PRECEDING) 
        else null
        end as depth3
    from aoc_day01)
, depth3_diff as (select depth3 - (lag(depth3,1) over(order by seq asc)) as diff
        from depth3
)
select count(*) as count 
    from depth3_diff where diff > 0

 * db2+ibm_db://db2inst1:***@localhost:50000/aocdb
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: SQLNumResultCols failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0104N  An unexpected token ")" was found following "from depth3)".  Expected tokens may include:  "<space>".  SQLSTATE=42601 SQLCODE=-104
[SQL: with depth3 as (select
    case when count(depth) over(order by seq asc ROWS 2 PRECEDING) = 3
        then sum(depth) over(order by seq asc ROWS 2 PRECEDING) 
        else null
        end as depth3
    from aoc_day01)
, depth3_diff as (select depth3 - (lag(depth3,1) over(order by seq asc)) as diff
        from depth3)
)
select count(*) as count 
    from depth3_diff where diff > 0]
(Background on this error at: http://sqlalche.me/e/f405)
